In [ ]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kalpana15mishra","key":"499cf673b08ea3c75c88c1f265df6505"}'}

In [ ]:
!pip install -q kaggle
import os

os.makedirs('/root/.kaggle', exist_ok=True)
!cp /content/kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d badasstechie/celebahq-resized-256x256
!unzip -q celebahq-resized-256x256.zip -d celeba_hq_256

Dataset URL: https://www.kaggle.com/datasets/badasstechie/celebahq-resized-256x256
License(s): GPL-2.0


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
import glob
import time
from IPython import display


In [ ]:
IMG_SIZE = 64
BATCH_SIZE = 64
BUFFER_SIZE = 1000
EPOCHS = 500
NOISE_DIM = 100
NUM_EXAMPLES_TO_GENERATE = 16

In [ ]:
def load_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    image = (image - 127.5) / 127.5
    return image

image_paths = glob.glob('/content/celeba_hq_256/**/*.jpg', recursive=True)
print("Total images found:", len(image_paths))


Total images found: 30000


In [ ]:
image_paths = image_paths[:5000]

dataset = tf.data.Dataset.from_tensor_slices(image_paths)
dataset = dataset.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [ ]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(8*8*256, use_bias=False, input_shape=(NOISE_DIM,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Reshape((8, 8, 256)))

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    return model

In [ ]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[IMG_SIZE, IMG_SIZE, 3]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))
    return model

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator = make_generator_model()
discriminator = make_discriminator_model()

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
seed = tf.random.normal([NUM_EXAMPLES_TO_GENERATE, NOISE_DIM])

def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    predictions = (predictions + 1) / 2.0

    fig = plt.figure(figsize=(4, 4))
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i])
        plt.axis('off')

    plt.savefig(f'image_at_epoch_{epoch:04d}.png')
    plt.close()

@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, NOISE_DIM])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

def train(dataset, epochs):
    for epoch in range(1, epochs + 1):
        start = time.time()

        for image_batch in dataset:
            train_step(image_batch)

        display.clear_output(wait=True)
        generate_and_save_images(generator, epoch, seed)
        print(f'Time for epoch {epoch} is {time.time() - start:.2f} sec')

In [ ]:
train(dataset, EPOCHS)

Time for epoch 500 is 5.03 sec


In [ ]:
!zip -r generated_images.zip *.png

  adding: image_at_epoch_0001.png (deflated 0%)
  adding: image_at_epoch_0002.png (deflated 0%)
  adding: image_at_epoch_0003.png (deflated 0%)
  adding: image_at_epoch_0004.png (deflated 0%)
  adding: image_at_epoch_0005.png (deflated 1%)
  adding: image_at_epoch_0006.png (deflated 1%)
  adding: image_at_epoch_0007.png (deflated 1%)
  adding: image_at_epoch_0008.png (deflated 1%)
  adding: image_at_epoch_0009.png (deflated 0%)
  adding: image_at_epoch_0010.png (deflated 0%)
  adding: image_at_epoch_0011.png (deflated 0%)
  adding: image_at_epoch_0012.png (deflated 1%)
  adding: image_at_epoch_0013.png (deflated 0%)
  adding: image_at_epoch_0014.png (deflated 0%)
  adding: image_at_epoch_0015.png (deflated 1%)
  adding: image_at_epoch_0016.png (deflated 0%)
  adding: image_at_epoch_0017.png (deflated 0%)
  adding: image_at_epoch_0018.png (deflated 0%)
  adding: image_at_epoch_0019.png (deflated 0%)
  adding: image_at_epoch_0020.png (deflated 1%)
  adding: image_at_epoch_0021.png (defla

In [ ]:
from google.colab import files
files.download("generated_images.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# ✅ STEP 11: Install dependencies (if not done)
!pip install -q scipy

# ✅ STEP 12: Imports
import numpy as np
from scipy import linalg
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.image import resize

# ✅ STEP 13: Load InceptionV3 without top layer
inception_model = InceptionV3(include_top=False, pooling='avg', input_shape=(299, 299, 3))
inception_model.trainable = False

# ✅ STEP 14: Feature extractor
def get_activations(images, batch_size=64):
    activations = []
    for i in range(0, images.shape[0], batch_size):
        batch = images[i:i+batch_size]
        batch = resize(batch, [299, 299])
        batch = preprocess_input(batch * 255.0)  # scale back to [0, 255] for preprocessing
        act = inception_model.predict(batch, verbose=0)
        activations.append(act)
    return np.concatenate(activations, axis=0)

# ✅ STEP 15: Compute statistics
def calculate_fid(real_acts, gen_acts):
    mu1, sigma1 = real_acts.mean(axis=0), np.cov(real_acts, rowvar=False)
    mu2, sigma2 = gen_acts.mean(axis=0), np.cov(gen_acts, rowvar=False)

    diff = mu1 - mu2
    covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)

    if np.iscomplexobj(covmean):
        covmean = covmean.real

    fid = diff.dot(diff) + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid

# ✅ STEP 16: Get real and fake images
NUM_FID_SAMPLES = 1000

# Sample real images
real_images = []
for img in dataset.unbatch().take(NUM_FID_SAMPLES):
    real_images.append(img)
real_images = tf.stack(real_images)

# Generate fake images
noise = tf.random.normal([NUM_FID_SAMPLES, NOISE_DIM])
fake_images = generator(noise, training=False)

# ✅ STEP 17: Get activations
real_acts = get_activations(real_images)
fake_acts = get_activations(fake_images)

# ✅ STEP 18: Calculate FID
fid_score = calculate_fid(real_acts, fake_acts)
print(f"✅ FID Score: {fid_score:.4f} (lower is better)")


87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
✅ FID Score: 32.3738 (lower is better)
